# Détection de Squats en Temps Réel

Ce notebook permet de tester la détection de squats en utilisant MediaPipe Pose.

## Fonctionnalités
- Détection de pose humaine en temps réel
- Calcul de l'angle du genou (hanche-genou-cheville)
- Comptage automatique des répétitions
- Feedback visuel en temps réel

## Paramètres de détection
- Angle minimum (position basse): 90°
- Angle maximum (position haute): 160°
- Côté analysé: gauche (par défaut)


In [ ]:
# Imports nécessaires
import sys
from pathlib import Path
import cv2
import numpy as np

# Ajouter le répertoire racine au path
project_root = Path().resolve().parent.parent.parent
sys.path.insert(0, str(project_root))

from exercises.squat.squat_detector import SquatDetector


In [ ]:
# Initialiser le détecteur de squats
detector = SquatDetector()

print("Détecteur de squats initialisé!")
print(f"Configuration: {detector.config}")


In [ ]:
# Initialiser la webcam
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    raise RuntimeError("Impossible d'ouvrir la webcam")

print("Webcam initialisée. Appuyez sur 'q' pour quitter, 'r' pour réinitialiser le compteur.")


In [ ]:
# Boucle principale de détection
try:
    while True:
        ret, frame = cap.read()
        
        if not ret:
            print("Erreur: Impossible de lire la frame")
            break
        
        # Détecter les squats
        annotated_frame, results = detector.detect(frame)
        
        # Afficher la frame
        cv2.imshow('Squat Detection', annotated_frame)
        
        # Gérer les touches
        key = cv2.waitKey(1) & 0xFF
        
        if key == ord('q'):
            print("Arrêt de la détection...")
            break
        elif key == ord('r'):
            detector.reset_counter()
            print("Compteur réinitialisé!")
        
        # Afficher les résultats
        if results.get('pose_detected') and results.get('angle') is not None:
            print(f"\rReps: {results['counter']} | Stage: {results['stage']} | Angle: {int(results['angle'])}°", 
                  end='', flush=True)

except KeyboardInterrupt:
    print("\nInterruption par l'utilisateur")
finally:
    # Nettoyer
    cap.release()
    cv2.destroyAllWindows()
    
    # Afficher les statistiques finales
    stats = detector.get_stats()
    print(f"\n\nStatistiques finales:")
    print(f"Total de répétitions: {stats['counter']}")
